# Learning Norse from scratch


This notebook is for you if 

* Have not worked with Norse before
* You are not familiar with [PyTorch](https://pytorch.org)
* You prefer visual explanations

# Agenda

1. Learning PyTorch 
  * Familiarize yourself with the deep learning library PyTorch
2. Learning about spiking neurons
  * Quickly understand what spiking neurons are and why they are relevant
3. Learning Norse
  * Simulate and visualise neurons in minutes!

## 1. Crawling before you can walk: PyTorch

Before we get to Norse, we need to cover some basics of PyTorch.

First, we need to install torch:

In [ ]:
# This is the CPU installation
!pip install torch --quiet
# Uncomment and use this line if you have a GPU!
#!pip3 install torch==1.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

We can now import torch!

In [6]:
import torch

## Working with Tensors

Now that we have PyTorch installed, we can create a vector:
$$
  v =
  \left[ {\begin{array}{cc}
    1 \\
    2  \\
  \end{array} } \right]
$$

In [13]:
v = torch.tensor([1, 2])
v

tensor([1, 2])

## Working with matrices

It turns out we can use the same notation to create matrices

$$
  m =
  \left[ {\begin{array}{cc}
    1 & 2\\
    3 & 4  \\
  \end{array} } \right]
$$

In [11]:
m = torch.tensor([[1, 2], [3, 4]])
m

tensor([[1, 2],
        [3, 4]])

You may wonder why it's called `torch.tensor`. That's because PyTorch calles vectors, matrices, and higher-order objects **tensors**. 

Below you can see the relationship between a **scalar** value, a **vector**, a **matrix**, and a **tensor** in 3D.

![](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fhadrienj.github.io%2Fassets%2Fimages%2F2.1%2Fscalar-vector-matrix-tensor.png&f=1&nofb=1)
&copy; [Hadrienj](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.1-Scalars-Vectors-Matrices-and-Tensors/)

In [ ]:
# Exercise 1: Create a tensor
#
# You now know how to create a vector and a matrix. Create a tensor with 2x2x2 numbers
# Can you make it look like the image above?

t = ...

## Inspecting a tensor

Understanding the dimensionality, called the **shape**, of a tensor is hugely important. **You should at all times know the shape of the tensor you are working with**.

Here is how you can see the shape of a tensor:

In [15]:
torch.tensor([[1, 2, 3], [4, 5, 6]]).shape

torch.Size([2, 3])

This tells us there are two dimensions containing two elements in the row and three elements in the column.

Note, you can find much more information about tensors here: https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

In [ ]:
# Exercise 2: Analyzing a tensor
#
# What is the shape of the tensor you created before?

...

## PyTorch modules

PyTorch consist of a number of **modules** that contain useful functionalities. Examples include:
* `torch.nn.Linear` - Linear mapping between modules
* `torch.nn.ReLU`   - Rectified Linear Unit
* `torch.nn.Sequential` - Put modules together in sequence

If you are unfamiliar with the concepts above, please read more in the PyTorch documentation here: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Assume that we would like to apply the `ReLU` to one of our tensors. That works like so:

In [18]:
module = torch.nn.ReLU()
module(torch.tensor([-1, 1]))

tensor([0, 1])

Or the `Linear` module, that maps a number of inputs to a number of outputs:

In [23]:
module = torch.nn.Linear(in_features=2, out_features=1)
module(torch.tensor([-1.0, 0.4]))

tensor([-0.2071], grad_fn=<AddBackward0>)

What just happened? Where did that number come from?
A linear module simply applies `ax + b`. We specified the `x` in our tensor, but where did the `a` and the `b` come from?

Torch creates there **parameters** automatically. As a user, we *can* specify them, but we don't have to. Here are the current weights and biases in the layer:

In [ ]:
module.weight

In [ ]:
module.bias

## Creating a PyTorch network

We are now ready to create a PyTorch network! You haven't seen the `Sequential` module yet, but perhaps you can figure out how to apply it anyway?

In [ ]:
# Exercise 3: Using PyTorch modules
#
# Use the Sequential module to create a network consisting of two modules:
#  - Linear(2, 3)
#  - ReLU()
# Documentation is available here: https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html#torch.nn.Sequential

module = ...

module(torch.tensor([-2, 3]))

## Spiking neurons

The `ReLU` unit you saw before was a neuron activation unit. However, it only applies a function *once*. Real neurons live for years and exist over a large period of time. That is exactly what makes them so computationally interesting, and that is what we will be modelling now.


![](images/spiking_neuron.png)

Our neurons receives a number of inputs and can choose to do two things: **not spike (0)** or **spike (1)**.

But, instead of doing it *once* like with the `ReLU` unit, we are doing it **over multiple timesteps**.

Here is a visualization of **how the neuron behaves internally in time** if we give two types of inputs: a constant input of 0.1 or a sawtooth input oscillating between -0.1 and 0.1:

![](https://ncskth.github.io/norse-rl/_images/li.gif)

Here is a visualization of **how the neuron behaves internally in time** if we give it constant inputs: 0, 0.1, and 0.3:


![](https://ncskth.github.io/norse-rl/_images/spikes.gif)

## Remembering state

You may have noticed that the next value of the neuron, say $v_t$, depends on the *previous* value $v_{t-1}$. We model this using **state**: a way to **remember what the neuron value was before**. Visually, it looks like this:



<table>
<thead>
<tr>
    <td><img src="images/neuron_state.png"/></td>
    <td><pre>
output, state = module(input, state)
</pre>
</td>
</tr>
    </thead>
</table>

We can now model this in Norse!

<h2 style="text-align: center; font-size: 400%"> Norse = PyTorch + ⚡️Spikes</h2>

Norse uses the exact same coding style as PyTorch, except, we use spiking neurons.

First, we need to install and import Norse:

In [31]:
!pip install norse --quiet
import norse

In [78]:
import ipywidgets as widgets
from ipywidgets import interact

import matplotlib.pyplot as plt

module = norse.torch.LIFCell()
# use interact decorator to decorate the function, so the function can receive the slide bar's value with parameter x.
@interact(x=widgets.FloatSlider(min=0.0, max=0.5, step=0.02, value=0.3))
def simulate(x=0.15):
    with torch.no_grad():
        s = None
        out, ss = [], []
        for i in range(100):
            z, s = module(torch.tensor([x]), s)
            out.append(out)
            ss.append(s)
        vs = torch.stack([s.v for s in ss]).squeeze()
        plt.figure(figsize=(14, 4))
        plt.ylim(0,1.05)
        plt.plot(vs)
        plt.show()

interactive(children=(FloatSlider(value=0.3, description='x', max=0.5, step=0.02), Output()), _dom_classes=('w…

One simple neuron model we can apply is called the **Leaky integrate-and-fire** model, or `LIF` for short:

In [32]:
module = norse.torch.LIF()

Before we provide it any data, we need to recall the *temporal* properties of our spiking neurons. We cannot just provide it with single values, we need to **provide a matrix of values**: one dimension being input, another being time. We saw above that an input of 0.3 should make it spike, let's try that:

In [ ]:
# Exercise 4: Generate data
#
# What is this code doing? Examine its shape and print out its contents
data = torch.ones((100, 1)) * 0.3

We are now ready to apply this to our neuron:

In [ ]:
output = module(data)

In [ ]:
# Exercise 5: Analyse module output
#
# Look at the `output` variable. What does it contain? Why?

## Scaling the simulation

Using `(100, 1)` 

## Visualizing the output

Notice how 